In [ ]:
!pip install transformers
!pip install underthesea
!pip install datasets

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Fine-tune PhoBERT với data 70:30

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
from transformers import TFAutoModelForMaskedLM
import re
import underthesea
import datasets


# Hàm chuẩn hoá câu
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)
def standardize_data(row):
    if not isinstance(row, str):
        return "không"
    # Xóa dấu chấm, phẩy, hỏi ở cuối câu
    row = re.sub(r"[\.,\?]+$-", "", row)
    # Xóa tất cả dấu chấm, phẩy, chấm phẩy, chấm thang, ... trong câu
    row = emoji_pattern.sub(r'', row)
    row = row.replace("http://", " ").replace("<unk>", " ")
    # row = row.replace("http://", " ").replace("(", " ").replace("=", " ")\
    #     .replace(".", " ") \
    #     .replace(";", ",").replace("“", " ") \
    #     .replace(":", ",").replace("”", " ") \
    #     .replace('"', ",").replace("'", " ") \
    #     .replace("!", ",").replace("?", " ") \
    #     .replace("-", ",").replace("?", " ") \
    #     .replace("/", ",")
    row = re.sub(r"\s+", " ", row) # Remove multiple spaces in content
    if row == "":
        return "không"
    row = underthesea.text_normalize(row)
    row = row.strip().lower()
    return row
model_checkpoint = "vinai/phobert-base"
model = TFAutoModelForMaskedLM.from_pretrained(model_checkpoint)
# text = "http:// Tôi làm gì <mask>?"
# print(standardize_data(text))

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
import pandas as pd
file_pd = pd.read_csv("/content/drive/MyDrive/combined_csv.csv", encoding='utf-8')
vaild_pd = pd.read_csv("/content/drive/MyDrive/Student_Ins.csv", encoding='utf-8')
file_pd = file_pd[['Bio_personality', 'food_drink', 'hobby_interests']]
from sklearn.model_selection import train_test_split
file_pd, test_pd = train_test_split(file_pd, test_size=0.3)
vaild_pd = vaild_pd[['Bio_personality ( tính cách cá nhân )', 'food_drink (Đồ ăn thức uống)', 'hobby_interests (sở thích cá nhân)']]
print(file_pd)
print(test_pd)
# file_text_pd = pd.DataFrame(file_pd.values.ravel('F'))
# file_text_pd.columns = ["text"]
file_pd = file_pd.stack().reset_index()
file_pd.columns = ["level_0", "level_1", "text"]
print(file_pd)
test_pd = test_pd.stack().reset_index()
test_pd.columns = ["level_0", "level_1", "text"]
print(test_pd)
vaild_pd = vaild_pd.stack().reset_index()
vaild_pd.columns = ["level_0", "level_1", "text"]

file_pd['text_cleaned'] = list(map(lambda x:standardize_data(x),file_pd['text']))
test_pd['text_cleaned'] = list(map(lambda x:standardize_data(x),test_pd['text']))
vaild_pd['text_cleaned'] = list(map(lambda x:standardize_data(x),vaild_pd['text']))


file_pd["labels"] = file_pd['text_cleaned'].copy()
test_pd["labels"] = test_pd['text_cleaned'].copy()
vaild_pd["labels"] = vaild_pd['text_cleaned'].copy()
print(len(test_pd['text_cleaned']))
def tokenize_function(examples):
    result = tokenizer(examples["text_cleaned"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

tdf = pd.DataFrame({'text_cleaned': file_pd['text_cleaned'].tolist(), "labels":file_pd["labels"].tolist()})
tds = datasets.Dataset.from_pandas(tdf)

test_df = pd.DataFrame({'text_cleaned': test_pd['text_cleaned'].tolist(), "labels":test_pd["labels"].tolist()})
test_df = datasets.Dataset.from_pandas(test_df)

valid_df = pd.DataFrame({'text_cleaned': vaild_pd['text_cleaned'].tolist(), "labels":vaild_pd["labels"].tolist()})
valid_df = datasets.Dataset.from_pandas(valid_df)
tds = datasets.DatasetDict({"train":tds, "test": test_df, "valid": valid_df})
tokenized_datasets = tds.map(
    tokenize_function, batched=True, remove_columns=["text_cleaned", "labels"]
)
print(tokenized_datasets)

from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15, return_tensors="tf")
data = [tokenized_datasets["train"][i] for i in range(len(tokenized_datasets["train"]))]
# input_ids = [chunk for chunk in data_collator(data)["input_ids"]]
# attention_masks  = tokenized_datasets["train"]["attention_mask"]
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    shuffle=True,
    batch_size=32,
    collate_fn=data_collator
)
tf_test_dataset = tokenized_datasets["test"].to_tf_dataset(
    shuffle=True,
    batch_size=32,
    collate_fn=data_collator
)
tf_valid_dataset = tokenized_datasets["valid"].to_tf_dataset(
    shuffle=True,
    batch_size=32,
    collate_fn=data_collator
)
from transformers import create_optimizer, AdamWeightDecay
from transformers.keras_callbacks import PushToHubCallback
import tensorflow as tf

num_train_steps = len(tf_train_dataset)
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=1_000,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
early_stopper = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                 patience=5, 
                                                 restore_best_weights=True)

# Train in mixed-precision float16
#tf.keras.mixed_precision.set_global_policy("mixed_float16")
model = TFAutoModelForMaskedLM.from_pretrained('dung1308/RM_system_not_mixed__NLP_model_90_10_CPU_2_epochs')
model_name = model_checkpoint.split("/")[-1]
push_to_hub_model_id = f"{model_name}-finetuned-vbert"
callback = PushToHubCallback(
    output_dir="/hello", tokenizer=tokenizer
)
#model.to(device)
earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min', restore_best_weights=True)
model.compile(optimizer=optimizer)
import math
eval_results = model.evaluate(tf_valid_dataset)
print(f"Perplexity: {math.exp(eval_results):.2f}")
history = model.fit(tf_train_dataset, validation_data=tf_test_dataset, epochs=2, callbacks = [earlyStopping,callback])
#history = model.fit(tf_train_dataset, validation_data=tf_test_dataset, epochs=2)
import math
eval_results = model.evaluate(tf_valid_dataset)
print(f"Perplexity: {math.exp(eval_results):.2f}")

with open('/content/drive/MyDrive/Perplexity_70_30.txt', 'w') as f:
    f.write(f"Perplexity of RM_system_NLP_not_mixed_float16_model_70_30: {math.exp(eval_results):.2f}")
from matplotlib import pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig('/content/drive/MyDrive/loss_not_mixed_float16_70_30.png')

print("Finish")

Fine-tune PhoBERT với data 80:20

In [ ]:
from transformers import TFAutoModelForMaskedLM
import re
import underthesea
import datasets


# Hàm chuẩn hoá câu
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)
def standardize_data(row):
    if not isinstance(row, str):
        return "không"
    # Xóa dấu chấm, phẩy, hỏi ở cuối câu
    row = re.sub(r"[\.,\?]+$-", "", row)
    # Xóa tất cả dấu chấm, phẩy, chấm phẩy, chấm thang, ... trong câu
    row = emoji_pattern.sub(r'', row)
    row = row.replace("http://", " ").replace("<unk>", " ")
    # row = row.replace("http://", " ").replace("(", " ").replace("=", " ")\
    #     .replace(".", " ") \
    #     .replace(";", ",").replace("“", " ") \
    #     .replace(":", ",").replace("”", " ") \
    #     .replace('"', ",").replace("'", " ") \
    #     .replace("!", ",").replace("?", " ") \
    #     .replace("-", ",").replace("?", " ") \
    #     .replace("/", ",")
    row = re.sub(r"\s+", " ", row) # Remove multiple spaces in content
    if row == "":
        return "không"
    row = underthesea.text_normalize(row)
    row = row.strip().lower()
    return row
model_checkpoint = "vinai/phobert-base"
model = TFAutoModelForMaskedLM.from_pretrained(model_checkpoint)
# text = "http:// Tôi làm gì <mask>?"
# print(standardize_data(text))

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
import pandas as pd
file_pd = pd.read_csv("/content/drive/MyDrive/combined_csv.csv", encoding='utf-8')
vaild_pd = pd.read_csv("/content/drive/MyDrive/Student_Ins.csv", encoding='utf-8')
file_pd = file_pd[['Bio_personality', 'food_drink', 'hobby_interests']]
from sklearn.model_selection import train_test_split
file_pd, test_pd = train_test_split(file_pd, test_size=0.2)
vaild_pd = vaild_pd[['Bio_personality ( tính cách cá nhân )', 'food_drink (Đồ ăn thức uống)', 'hobby_interests (sở thích cá nhân)']]
print(file_pd)
print(test_pd)
# file_text_pd = pd.DataFrame(file_pd.values.ravel('F'))
# file_text_pd.columns = ["text"]
file_pd = file_pd.stack().reset_index()
file_pd.columns = ["level_0", "level_1", "text"]
print(file_pd)
test_pd = test_pd.stack().reset_index()
test_pd.columns = ["level_0", "level_1", "text"]
print(test_pd)
vaild_pd = vaild_pd.stack().reset_index()
vaild_pd.columns = ["level_0", "level_1", "text"]

file_pd['text_cleaned'] = list(map(lambda x:standardize_data(x),file_pd['text']))
test_pd['text_cleaned'] = list(map(lambda x:standardize_data(x),test_pd['text']))
vaild_pd['text_cleaned'] = list(map(lambda x:standardize_data(x),vaild_pd['text']))


file_pd["labels"] = file_pd['text_cleaned'].copy()
test_pd["labels"] = test_pd['text_cleaned'].copy()
vaild_pd["labels"] = vaild_pd['text_cleaned'].copy()
print(len(test_pd['text_cleaned']))
def tokenize_function(examples):
    result = tokenizer(examples["text_cleaned"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

tdf = pd.DataFrame({'text_cleaned': file_pd['text_cleaned'].tolist(), "labels":file_pd["labels"].tolist()})
tds = datasets.Dataset.from_pandas(tdf)

test_df = pd.DataFrame({'text_cleaned': test_pd['text_cleaned'].tolist(), "labels":test_pd["labels"].tolist()})
test_df = datasets.Dataset.from_pandas(test_df)

valid_df = pd.DataFrame({'text_cleaned': vaild_pd['text_cleaned'].tolist(), "labels":vaild_pd["labels"].tolist()})
valid_df = datasets.Dataset.from_pandas(valid_df)
tds = datasets.DatasetDict({"train":tds, "test": test_df, "valid": valid_df})
tokenized_datasets = tds.map(
    tokenize_function, batched=True, remove_columns=["text_cleaned", "labels"]
)
print(tokenized_datasets)

from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15, return_tensors="tf")
data = [tokenized_datasets["train"][i] for i in range(len(tokenized_datasets["train"]))]
# input_ids = [chunk for chunk in data_collator(data)["input_ids"]]
# attention_masks  = tokenized_datasets["train"]["attention_mask"]
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    shuffle=True,
    batch_size=32,
    collate_fn=data_collator
)
tf_test_dataset = tokenized_datasets["test"].to_tf_dataset(
    shuffle=True,
    batch_size=32,
    collate_fn=data_collator
)
tf_valid_dataset = tokenized_datasets["valid"].to_tf_dataset(
    shuffle=True,
    batch_size=32,
    collate_fn=data_collator
)
from transformers import create_optimizer, AdamWeightDecay
from transformers.keras_callbacks import PushToHubCallback
import tensorflow as tf

num_train_steps = len(tf_train_dataset)
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=1_000,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
early_stopper = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                 patience=5, 
                                                 restore_best_weights=True)

# Train in mixed-precision float16
#tf.keras.mixed_precision.set_global_policy("mixed_float16")
model_name = model_checkpoint.split("/")[-1]
push_to_hub_model_id = f"{model_name}-finetuned-vbert"
callback = PushToHubCallback(
    output_dir="/content/drive/MyDrive/RM_system_not_mixed__NLP_model_80_20_Colab_2epochs", tokenizer=tokenizer
)
#model.to(device)
earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min', restore_best_weights=True)
model.compile(optimizer=optimizer)
import math
eval_results = model.evaluate(tf_valid_dataset)
print(f"Perplexity: {math.exp(eval_results):.2f}")
history = model.fit(tf_train_dataset, validation_data=tf_test_dataset, epochs=20, callbacks=[earlyStopping, callback])
# history = model.fit(tf_train_dataset, validation_data=tf_test_dataset, epochs=2,callbacks=[callback])

import math
eval_results = model.evaluate(tf_valid_dataset)
print(f"Perplexity: {math.exp(eval_results):.2f}")

with open('/content/drive/MyDrive/Perplexity_80_20.txt', 'w') as f:
    f.write(f"Perplexity of RM_system_NLP_not_mixed_float16_model_70_30: {math.exp(eval_results):.2f}")
from matplotlib import pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig('/content/drive/MyDrive/loss_not_mixed_float16_80_20.png')

Fine-tune PhoBERT với data 90:10

In [ ]:
from transformers import TFAutoModelForMaskedLM
import re
import underthesea
import datasets

# Hàm chuẩn hoá câu
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)
def standardize_data(row):
    if not isinstance(row, str):
        return "không"
    # Xóa dấu chấm, phẩy, hỏi ở cuối câu
    row = re.sub(r"[\.,\?]+$-", "", row)
    # Xóa tất cả dấu chấm, phẩy, chấm phẩy, chấm thang, ... trong câu
    row = emoji_pattern.sub(r'', row)
    row = row.replace("http://", " ").replace("<unk>", " ")
    # row = row.replace("http://", " ").replace("(", " ").replace("=", " ")\
    #     .replace(".", " ") \
    #     .replace(";", ",").replace("“", " ") \
    #     .replace(":", ",").replace("”", " ") \
    #     .replace('"', ",").replace("'", " ") \
    #     .replace("!", ",").replace("?", " ") \
    #     .replace("-", ",").replace("?", " ") \
    #     .replace("/", ",")
    row = re.sub(r"\s+", " ", row) # Remove multiple spaces in content
    if row == "":
        return "không"
    row = underthesea.text_normalize(row)
    row = row.strip().lower()
    return row
model_checkpoint = "vinai/phobert-base"
model = TFAutoModelForMaskedLM.from_pretrained(model_checkpoint)
# text = "http:// Tôi làm gì <mask>?"
# print(standardize_data(text))

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
import pandas as pd
file_pd = pd.read_csv("/content/sample_data/combined_csv.csv", encoding='utf-8')
vaild_pd = pd.read_csv("/content/sample_data/Student_Ins.csv", encoding='utf-8')
file_pd = file_pd[['Bio_personality', 'food_drink', 'hobby_interests']]
from sklearn.model_selection import train_test_split
file_pd, test_pd = train_test_split(file_pd, test_size=0.1)
vaild_pd = vaild_pd[['Bio_personality ( tính cách cá nhân )', 'food_drink (Đồ ăn thức uống)', 'hobby_interests (sở thích cá nhân)']]
print(file_pd)
print(test_pd)
# file_text_pd = pd.DataFrame(file_pd.values.ravel('F'))
# file_text_pd.columns = ["text"]
file_pd = file_pd.stack().reset_index()
file_pd.columns = ["level_0", "level_1", "text"]
print(file_pd)
test_pd = test_pd.stack().reset_index()
test_pd.columns = ["level_0", "level_1", "text"]
print(test_pd)
vaild_pd = vaild_pd.stack().reset_index()
vaild_pd.columns = ["level_0", "level_1", "text"]

file_pd['text_cleaned'] = list(map(lambda x:standardize_data(x),file_pd['text']))
test_pd['text_cleaned'] = list(map(lambda x:standardize_data(x),test_pd['text']))
vaild_pd['text_cleaned'] = list(map(lambda x:standardize_data(x),vaild_pd['text']))


file_pd["labels"] = file_pd['text_cleaned'].copy()
test_pd["labels"] = test_pd['text_cleaned'].copy()
vaild_pd["labels"] = vaild_pd['text_cleaned'].copy()
print(len(test_pd['text_cleaned']))
def tokenize_function(examples):
    result = tokenizer(examples["text_cleaned"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

tdf = pd.DataFrame({'text_cleaned': file_pd['text_cleaned'].tolist(), "labels":file_pd["labels"].tolist()})
tds = datasets.Dataset.from_pandas(tdf)

test_df = pd.DataFrame({'text_cleaned': test_pd['text_cleaned'].tolist(), "labels":test_pd["labels"].tolist()})
test_df = datasets.Dataset.from_pandas(test_df)

valid_df = pd.DataFrame({'text_cleaned': vaild_pd['text_cleaned'].tolist(), "labels":vaild_pd["labels"].tolist()})
valid_df = datasets.Dataset.from_pandas(valid_df)
tds = datasets.DatasetDict({"train":tds, "test": test_df, "valid": valid_df})
tokenized_datasets = tds.map(
    tokenize_function, batched=True, remove_columns=["text_cleaned", "labels"]
)
print(tokenized_datasets)

from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15, return_tensors="tf")
data = [tokenized_datasets["train"][i] for i in range(len(tokenized_datasets["train"]))]
# input_ids = [chunk for chunk in data_collator(data)["input_ids"]]
# attention_masks  = tokenized_datasets["train"]["attention_mask"]
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    shuffle=True,
    batch_size=32,
    collate_fn=data_collator
)
tf_test_dataset = tokenized_datasets["test"].to_tf_dataset(
    shuffle=True,
    batch_size=32,
    collate_fn=data_collator
)
tf_valid_dataset = tokenized_datasets["valid"].to_tf_dataset(
    shuffle=True,
    batch_size=32,
    collate_fn=data_collator
)
from transformers import create_optimizer, AdamWeightDecay
from transformers.keras_callbacks import PushToHubCallback
import tensorflow as tf

num_train_steps = len(tf_train_dataset)
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=1_000,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
early_stopper = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                 patience=5, 
                                                 restore_best_weights=True)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")
model_name = model_checkpoint.split("/")[-1]
push_to_hub_model_id = f"{model_name}-finetuned-vbert"
callback = PushToHubCallback(
    output_dir="./RM_system_not_mixed__NLP_model_90_10_Colab", tokenizer=tokenizer
)
#model.to(device)
earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min', restore_best_weights=True)
model.compile(optimizer=optimizer)
import math
eval_results = model.evaluate(tf_valid_dataset)
print(f"Perplexity: {math.exp(eval_results):.2f}")
history = model.fit(tf_train_dataset, validation_data=tf_test_dataset, epochs=20, callbacks=[earlyStopping, callback])
#history = model.fit(tf_train_dataset, validation_data=tf_test_dataset, epochs=2, callbacks=[callback])
import math
eval_results = model.evaluate(tf_valid_dataset)
print(f"Perplexity: {math.exp(eval_results):.2f}")

with open('/content/Perplexity_90_10.txt', 'w') as f:
    f.write(f"Perplexity of RM_system_NLP_not_mixed_float16_model_90_10: {math.exp(eval_results):.2f}")
from matplotlib import pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig('/content/loss_not_mixed_float16_90_10.png')

In [ ]:


!zip -r /content/file.zip /content/Folder_To_Zip

Fine-tuning 60:40

In [ ]:
from transformers import TFAutoModelForMaskedLM
import re
import underthesea
import datasets


# Hàm chuẩn hoá câu
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)
def standardize_data(row):
    if not isinstance(row, str):
        return "không"
    # Xóa dấu chấm, phẩy, hỏi ở cuối câu
    row = re.sub(r"[\.,\?]+$-", "", row)
    # Xóa tất cả dấu chấm, phẩy, chấm phẩy, chấm thang, ... trong câu
    row = emoji_pattern.sub(r'', row)
    row = row.replace("http://", " ").replace("<unk>", " ")
    # row = row.replace("http://", " ").replace("(", " ").replace("=", " ")\
    #     .replace(".", " ") \
    #     .replace(";", ",").replace("“", " ") \
    #     .replace(":", ",").replace("”", " ") \
    #     .replace('"', ",").replace("'", " ") \
    #     .replace("!", ",").replace("?", " ") \
    #     .replace("-", ",").replace("?", " ") \
    #     .replace("/", ",")
    row = re.sub(r"\s+", " ", row) # Remove multiple spaces in content
    if row == "":
        return "không"
    row = underthesea.text_normalize(row)
    row = row.strip().lower()
    return row
model_checkpoint = "vinai/phobert-base"
model = TFAutoModelForMaskedLM.from_pretrained(model_checkpoint)
# text = "http:// Tôi làm gì <mask>?"
# print(standardize_data(text))

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
import pandas as pd
file_pd = pd.read_csv("/content/drive/MyDrive/combined_csv.csv", encoding='utf-8')
vaild_pd = pd.read_csv("/content/drive/MyDrive/Student_Ins.csv", encoding='utf-8')
file_pd = file_pd[['Bio_personality', 'food_drink', 'hobby_interests']]
from sklearn.model_selection import train_test_split
file_pd, test_pd = train_test_split(file_pd, test_size=0.4)
vaild_pd = vaild_pd[['Bio_personality ( tính cách cá nhân )', 'food_drink (Đồ ăn thức uống)', 'hobby_interests (sở thích cá nhân)']]
print(file_pd)
print(test_pd)
# file_text_pd = pd.DataFrame(file_pd.values.ravel('F'))
# file_text_pd.columns = ["text"]
file_pd = file_pd.stack().reset_index()
file_pd.columns = ["level_0", "level_1", "text"]
print(file_pd)
test_pd = test_pd.stack().reset_index()
test_pd.columns = ["level_0", "level_1", "text"]
print(test_pd)
vaild_pd = vaild_pd.stack().reset_index()
vaild_pd.columns = ["level_0", "level_1", "text"]

file_pd['text_cleaned'] = list(map(lambda x:standardize_data(x),file_pd['text']))
test_pd['text_cleaned'] = list(map(lambda x:standardize_data(x),test_pd['text']))
vaild_pd['text_cleaned'] = list(map(lambda x:standardize_data(x),vaild_pd['text']))


file_pd["labels"] = file_pd['text_cleaned'].copy()
test_pd["labels"] = test_pd['text_cleaned'].copy()
vaild_pd["labels"] = vaild_pd['text_cleaned'].copy()
print(len(test_pd['text_cleaned']))
def tokenize_function(examples):
    result = tokenizer(examples["text_cleaned"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

tdf = pd.DataFrame({'text_cleaned': file_pd['text_cleaned'].tolist(), "labels":file_pd["labels"].tolist()})
tds = datasets.Dataset.from_pandas(tdf)

test_df = pd.DataFrame({'text_cleaned': test_pd['text_cleaned'].tolist(), "labels":test_pd["labels"].tolist()})
test_df = datasets.Dataset.from_pandas(test_df)

valid_df = pd.DataFrame({'text_cleaned': vaild_pd['text_cleaned'].tolist(), "labels":vaild_pd["labels"].tolist()})
valid_df = datasets.Dataset.from_pandas(valid_df)
tds = datasets.DatasetDict({"train":tds, "test": test_df, "valid": valid_df})
tokenized_datasets = tds.map(
    tokenize_function, batched=True, remove_columns=["text_cleaned", "labels"]
)
print(tokenized_datasets)

from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15, return_tensors="tf")
data = [tokenized_datasets["train"][i] for i in range(len(tokenized_datasets["train"]))]
# input_ids = [chunk for chunk in data_collator(data)["input_ids"]]
# attention_masks  = tokenized_datasets["train"]["attention_mask"]
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    shuffle=True,
    batch_size=32,
    collate_fn=data_collator
)
tf_test_dataset = tokenized_datasets["test"].to_tf_dataset(
    shuffle=True,
    batch_size=32,
    collate_fn=data_collator
)
tf_valid_dataset = tokenized_datasets["valid"].to_tf_dataset(
    shuffle=True,
    batch_size=32,
    collate_fn=data_collator
)
from transformers import create_optimizer, AdamWeightDecay
from transformers.keras_callbacks import PushToHubCallback
import tensorflow as tf

num_train_steps = len(tf_train_dataset)
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=1_000,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
early_stopper = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                 patience=5, 
                                                 restore_best_weights=True)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")
model_name = model_checkpoint.split("/")[-1]
push_to_hub_model_id = f"{model_name}-finetuned-vbert"
callback = PushToHubCallback(
    output_dir="/content/drive/MyDrive/RM_system_not_mixed__NLP_model_60_40_Colab_2epochs", tokenizer=tokenizer
)
#model.to(device)
earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min', restore_best_weights=True)
model.compile(optimizer=optimizer)
import math
eval_results = model.evaluate(tf_valid_dataset)
print(f"Perplexity: {math.exp(eval_results):.2f}")
history = model.fit(tf_train_dataset, validation_data=tf_test_dataset, epochs=20, callbacks=[earlyStopping, callback])
#history = model.fit(tf_train_dataset, validation_data=tf_test_dataset, epochs=2,callbacks=[callback])
import math
eval_results = model.evaluate(tf_valid_dataset)
print(f"Perplexity: {math.exp(eval_results):.2f}")
with open('/content/Perplexity_60_40.txt', 'w') as f:
    f.write(f"Perplexity of RM_system_NLP_not_mixed_float16_model_60_40: {math.exp(eval_results):.2f}")
from matplotlib import pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig('/content/loss_not_mixed_float16_60_40.png')